# Text generation using a RNN

# 1.Goal

Given a word, or a sequence of words, what is the most probable next word? This is the task we're training the model to perform. The input to the model will be a sequence of words, and we train the model to predict the output—the following word at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the words computed until this moment, what is the next word?

Libraries used in this notebook along with their version:

google	2.0.3

numpy	1.18.3

tensorflow	2.4.1

sklearn 0.0

Given a sequence of words from this data, train a model to predict the next word in the sequence. Longer sequences of text can be generated by calling the model repeatedly.

In [ ]:
import tensorflow
tensorflow.__version__

'2.4.1'

**Mount your Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/My_NLP/Session 4/FNB/')

# 2.Import Keras and other libraries

In [ ]:
import glob

from sklearn.utils import shuffle
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend

# 3.Download data
Data is collected from http://www.gutenberg.org


### Load the Tagore dataset

Store all the ".txt" file names in a list

In [ ]:
import zipfile

In [ ]:
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/My_NLP/Session 4/FNB/Tagore.zip", 'r')
zip_ref.extractall("/content/drive/MyDrive/My_NLP/Session 4/FNB/Tagore")
zip_ref.close()

In [ ]:
filelist = glob.glob("/content/drive/MyDrive/My_NLP/Session 4/FNB/Tagore/data/*.txt")

### Read the data

Read contents of every file from the list and append the text in a new list

In [ ]:
text_data = []
for file in filelist:
    with open(file, "r") as file:
      text_data.append(file.read())

# 4.Process the text
Initialize and fit the tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping words to numbers, and another for numbers to words.

In [ ]:
word_idx = tokenizer.word_index #Last is the key
idx_word = tokenizer.index_word

Get the word count for every word and also get the total number of words.

In [ ]:
word_counts = tokenizer.word_counts
num_words = len(word_counts)

Convert text to sequence of numbers

In [ ]:
sequences = tokenizer.texts_to_sequences(text_data)

### Generate Features and Labels

In [ ]:
features = []
labels = []

training_length = 50
# Iterate through the sequences of tokens
for seq in sequences:
    # Create multiple training examples from each sequence
    for i in range(training_length, training_length+300):
        # Extract the features and label
        extract = seq[i - training_length: i - training_length + 20]

        # Set the features and label
        features.append(extract[:-1])
        labels.append(extract[-1])

# **The prediction task**

# 5.Generate training and testing data

In [ ]:
from sklearn.utils import shuffle
import numpy as np

features, labels = shuffle(features, labels, random_state=1)

# Decide on number of samples for training
train_end = int(0.75 * len(labels))

train_features = np.array(features[:train_end])
valid_features = np.array(features[train_end:])

train_labels = labels[:train_end]
valid_labels = labels[train_end:]

# Convert to arrays
X_train, X_valid = np.array(train_features), np.array(valid_features)

# Using int8 for memory savings
y_train = np.zeros((len(train_labels), num_words), dtype=np.int8)
y_valid = np.zeros((len(valid_labels), num_words), dtype=np.int8)

# One hot encoding of labels
for example_index, word_index in enumerate(train_labels):
    y_train[example_index, word_index] = 1

for example_index, word_index in enumerate(valid_labels):
    y_valid[example_index, word_index] = 1

This is just to check the features and labels

In [ ]:
for i, sequence in enumerate(X_train[:1]):
    text = []
    for idx in sequence:
        text.append(idx_word[idx])
        
    print('Features: ' + ' '.join(text)+'\n')
    print('Label: ' + idx_word[np.argmax(y_train[i])] + '\n')

Features: herself completely with her full burden of sweetness iii is summer's festival only for fresh blossoms and not also

Label: for



# 6.Build The Model

Use `keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `keras.layers.LSTM`: A type of RNN with size `units=rnn_units` (You can also use a GRU layer here.)
* `keras.layers.Dense`: The output layer, with `num_words` outputs.

In [ ]:
model = Sequential()

# Embedding layer
model.add(
    Embedding(
        input_dim=num_words,
        output_dim=100,
        weights=None,
        trainable=True))

# Recurrent layer
model.add(
    LSTM(
        64, return_sequences=False, dropout=0.1,
        recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         2459500   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                42240     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 24595)             1598675   
Total params: 4,104,575
Trainable params: 4,104,575
Non-trainable params: 0
_________________________________________________________________


For each word the model looks up the embedding, runs the LSTM one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-liklihood of the next word.

# 7.Train the model

In [ ]:
h = model.fit(X_train, y_train, epochs = 1, batch_size = 100, verbose = 1)

45/45 [==============================] - 8s 136ms/step - loss: 9.8823 - accuracy: 0.0454


# 8.Save Model

In [ ]:
# save the model to file
model.save('./Tagore/data/model_1000epochs.h5')

## If you have already trained the model and saved it, you can load a pretrained model

In [ ]:
1
# load the model
2
model = load_model('./Tagore/data/model_1000epochs.h5')

### Note: After loading the model run  model.fit()  to continue training form there, if required.

In [ ]:
model.fit(X_train, y_train, batch_size=50, epochs=1000)

Epoch 1/1000
90/90 [==============================] - 10s 96ms/step - loss: 6.7267 - accuracy: 0.0600
Epoch 2/1000
90/90 [==============================] - 8s 94ms/step - loss: 6.4011 - accuracy: 0.0607
Epoch 3/1000
90/90 [==============================] - 8s 93ms/step - loss: 6.2273 - accuracy: 0.0609
Epoch 4/1000
90/90 [==============================] - 8s 92ms/step - loss: 6.1065 - accuracy: 0.0611
Epoch 5/1000
90/90 [==============================] - 8s 94ms/step - loss: 6.0221 - accuracy: 0.0613
Epoch 6/1000
90/90 [==============================] - 8s 93ms/step - loss: 5.9456 - accuracy: 0.0616
Epoch 7/1000
90/90 [==============================] - 8s 91ms/step - loss: 5.8913 - accuracy: 0.0653
Epoch 8/1000
90/90 [==============================] - 8s 92ms/step - loss: 5.8206 - accuracy: 0.0658
Epoch 9/1000
90/90 [==============================] - 8s 92ms/step - loss: 5.7606 - accuracy: 0.0691
Epoch 10/1000
90/90 [==============================] - 8s 92ms/step - loss: 5.6832 - accur

# 9.Evaluation

In [ ]:
print(model.evaluate(X_train, y_train, batch_size = 20))
print('\nModel Performance: Log Loss and Accuracy on validation data')
print(model.evaluate(X_valid, y_valid, batch_size = 20))

225/225 [==============================] - 4s 14ms/step - loss: 0.0105 - accuracy: 0.9960
[0.010506200604140759, 0.9959999918937683]

Model Performance: Log Loss and Accuracy on validation data
75/75 [==============================] - 1s 15ms/step - loss: 33.8489 - accuracy: 0.3767
[33.848934173583984, 0.3766666650772095]


# 10.Generate text

In [ ]:
seed_length=20
new_words=20
diversity=1
n_gen=1

import random

# Choose a random sequence
seq = random.choice(sequences)

# print seq

# Choose a random starting point
seed_idx = random.randint(0, len(seq) - seed_length - 10)
# Ending index for seed
end_idx = seed_idx + seed_length

gen_list = []

for n in range(n_gen):
    # Extract the seed sequence
    seed = seq[seed_idx:end_idx]
    original_sequence = [idx_word[i] for i in seed]
    generated = seed[:] + ['#']

    # Find the actual entire sequence
    actual = generated[:] + seq[end_idx:end_idx + new_words]
        
    # Keep adding new words
    for i in range(new_words):

        # Make a prediction from the seed
        preds = model.predict(np.array(seed).reshape(1, -1))[0].astype(np.float64)

  
        # Softmax
        preds = preds / sum(preds)

        # Choose the next word
        probas = np.random.multinomial(1, preds, 1)[0]

        next_idx = np.argmax(probas)

        # New seed adds on old word
        #             seed = seed[1:] + [next_idx]
        seed += [next_idx]
        generated.append(next_idx)
    # Showing generated and actual abstract
    n = []

    for i in generated:
        n.append(idx_word.get(i, '< --- >'))

    gen_list.append(n)

a = []

for i in actual:
    a.append(idx_word.get(i, '< --- >'))

a = a[seed_length:]

gen_list = [gen[seed_length:seed_length + len(a)] for gen in gen_list]

print('Original Sequence: \n'+' '.join(original_sequence))
print("\n")
# print(gen_list)
print('Generated Sequence: \n'+' '.join(gen_list[0][1:]))
# print(a)

Original Sequence: 
noblest heritage of truth lxv may be there is one house in this city where the gate opens for ever


Generated Sequence: 
known not been philosophically red hem and your wonderful eyes of chitravahana me her growth should no burden to me
